#Quantum Hangman



In [13]:
!pip install qiskit


# <ins>Method-I : Using Grover's Algoritm </ins>

- Firstly, to implemnet hangman game using above, we have to treat each letter of the given word as a quantum state.
- As in the hangman game, a player have to guess one letter among 26 english alphabets, we need atleast 26 states. But as the quantum states appears in numbers which are integer exponent of 2 (E.g. N= 2,4,8,16,....., because of the binary feature of quantum states taking values either 0 or 1), we have to consider 2^5=32 states, which would appear with letter_to_index conversion as 0 to 25 in python code for a to z (I would take .lower() for Capital letter input) and for 26 to 31 indices, I would opt for padding.
- The inherent idea is , if the player is able to guess correct letter, then the state associated with that letter, would be inverted and amplified using Grover's algorithm. Because of which, with proper number of iterations of amplification, we can get maximum probability of the correctly guessed state and that would be revealed for all it's ocuurences.  But for the wrong guess though the state would be inverted, but there would be no amplification and no word would be revealed.
- The _build_diffuser_circuit() function will be the diffuser part amplifying the correctly gussed state using the reflection of the inverted state about the mean of all states.
- The _backend_grover_sample() function will implement a quantum circuit for every guess and combine the state inverting Oracle part and Diffuser function. Besides that, it runs the quantum circuit and  transforms the states from bit-string binary format to index format using binary to decimal conversion and the registers the individual counts in a dictionary.
- The quantum_measure() function keeps an track on which letters have been chosen, and whether the current guess is correct or not. Then it decides iteration number for diffuser amplification using formula,
  
$$
iter  \approx
\begin{cases}
  \frac{\pi}{4} \times \sqrt{\frac{N}{ M}}, & \text{if } M=1 \\
  0 ,   & \text{if } M=0
\end{cases}
$$


  

  where M can take value either 1 (correct guess) or 0 (wrong guess). Then it will calculate the probabilities for individual appearances by their number counts.
- The _print_counts_descending() function would print the individual count and their probability after performing it all in main function, where we can see the maximum probability is coming for the right guess.


In [14]:
WORDS = ["qubit", "qubits", "superposition", "entanglement", "decoherence", "measurement",
    "interference", "phase", "amplitude", "statevector", "densitymatrix", "mixedstate",
    "purestate", "Hilbertspace", "ket", "bra", "braket", "observable", "operator",
    "unitary", "hermitian", "eigenstate", "eigenvalue", "basis", "computationalbasis",
    "Blochsphere", "Blochvector", "tensorproduct", "innerproduct", "outerproduct",
    "normalization", "probabilityamplitude", "probabilitydistribution", "postselection",
    "projectivemeasurement", "POVM", "ancilla", "ancillaryqubit", "classicalregister", "gate", "gates", "Hadamard", "Hgate", "PauliX", "PauliY", "PauliZ",
    "Xgate", "Ygate", "Zgate", "Sgate", "Tgate", "CNOT", "CX", "CZ",
    "Toffoli", "Fredkin", "Swap", "ControlledU", "multicontrol", "phasegate",
    "rotation", "Rx", "Ry", "Rz", "U3", "U2", "U1", "identitygate", "resetgate",
    "measure", "readout", "entanglinggate", "twoqubitgate", "threequbitgate",
    "parametricgate", "parametrizedcircuit", "Grover", "Groversearch", "Shor", "Shorfactorization", "QFT", "quantumfouriertransform",
    "phaseestimation", "quantumphaseestimation", "VQE", "variationalquantumeigensolver",
    "QAOA", "quantumapproxoptimization", "HHL", "harrowhassidimlloyd", "quantumwalk",
    "quantumsimulation", "adiabatic", "adiabaticquantumcomputation", "quantumchemistry",
    "quantummachinelearning", "qml", "quantumannealing", "amplitudeamplification",
    "errormitigation", "quantumkeydistribution", "QKD", "teleportation", "densecoding", "quantumerrorcorrection", "QEC", "surfacecode", "toriccode", "stabilizer",
    "stabilizercode", "syndrome", "syndromemeasurement", "logicalqubit", "physicalqubit",
    "faulttolerance", "threshold", "concatenatedcode", "CSScode", "Steanecode",
    "Shorcode", "distance", "errorrate", "bitflip", "phaseflip", "depolarizing",
    "amplitudedamping", "phasedamping", "leakage", "errormodel"]




In [15]:
#!/usr/bin/env python3
"""
Quantum Hangman (Grover) — assume a BasicProvider "basic_simulator" exists.

Behavior:
- Marks only the guessed letter when the guess is correct.
- Post-processes the measured histogram so the guessed letter becomes the
  most probable outcome (if correct).
- Prints the measured counts/probabilities as letter-labeled entries in
  descending order by count (letters first, then padding states).
"""
import random
import math
import numpy as np

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit.library import DiagonalGate
from qiskit.providers.basic_provider import BasicProvider


#Words list are chosen from above
#WORDS = ["quantum", "superposition", "entanglement", "measurement", "probability", "spin"]

class QuantumHangmanGrover:
    def __init__(self, word_list, shots=200):
        self.word = random.choice(word_list).lower()
        self.display = ['_' for _ in self.word]
        self.guessed_letters = set()
        self.max_tries = len(set(self.word))+6
        self.tries_left = self.max_tries
        self.shots = shots

        # Alphabet mapping: a->0, b->1, ..., z->25
        self.alphabet = [chr(ord('a') + i) for i in range(26)]
        self.letter_to_index = {ch: i for i, ch in enumerate(self.alphabet)}
        self.n_qubits = 5  # 2^5 = 32 >= 26
        self.N = 2 ** self.n_qubits

        # Assumption: BasicProvider can be instantiated and provides basic_simulator
        provider = BasicProvider()
        self.backend = provider.get_backend('basic_simulator')
        self.backend_name = "BasicProvider:basic_simulator"

        print(f"\nQuantum Hangman (Grover) initialized. The word has {len(self.word)} letters. You have {self.max_tries} tries.")
        print(f"Using provider backend: {self.backend_name}")

    def display_state(self):
        print("Word:", ' '.join(self.display))
        print(f"Guessed letters: {', '.join(sorted(self.guessed_letters))}")
        print(f"Tries remaining: {self.tries_left} (of original {self.max_tries})")

    def _build_diffuser_circuit(self):
        """
        Build diffuser as a QuantumCircuit acting on n_qubits and return it.
        Diffuser: H - X - (phase flip on |0...0>) - X - H
        """
        n = self.n_qubits
        diffuser = QuantumCircuit(n, name='diffuser')
        diffuser.h(range(n))
        diffuser.x(range(n))
        diag = np.ones(2**n, dtype=complex)
        diag[0] = -1.0
        diffuser.append(DiagonalGate(diag), list(range(n)))
        diffuser.x(range(n))
        diffuser.h(range(n))
        return diffuser

    def _backend_grover_sample(self, marked_indices, iterations, shots):
        """
        Build a Qiskit QuantumCircuit for Grover that marks only the indices in
        marked_indices, transpile it for self.backend, run it, and return
        counts_by_index (index -> integer count). Assumes backend is compatible.
        """
        n = self.n_qubits
        qreg = QuantumRegister(n, 'q')
        creg = ClassicalRegister(n, 'c')
        qc = QuantumCircuit(qreg, creg)

        # initialize uniform superposition
        qc.h(range(n))

        # build oracle diagonal: -1 on marked indices (if any)
        diag = np.ones(self.N, dtype=complex)
        for idx in marked_indices:
            diag[idx] = -1.0
        oracle_gate = DiagonalGate(diag)

        diffuser_gate = self._build_diffuser_circuit().to_gate()

        # apply Grover iterations (oracle + diffuser)
        for _ in range(iterations):
            qc.append(oracle_gate, list(range(n)))
            qc.append(diffuser_gate, list(range(n)))

        # measure all qubits
        qc.measure(range(n), range(n))

        # transpile and run on provided backend
        compiled = transpile(qc, self.backend)
        job = self.backend.run(compiled, shots=shots)
        result = job.result()
        raw_counts = result.get_counts()

        # convert bitstrings to integer indices (reverse bitstring for little-endian safety)
        counts_by_index = {}
        for bitstr, cnt in raw_counts.items():
            idx = int(bitstr[::-1], 2)
            counts_by_index[idx] = counts_by_index.get(idx, 0) + cnt
        return counts_by_index

    def _print_counts_descending(self, counts_by_index, probs_by_index):
        """
        Print letter-indexed counts (a..z) sorted by count descending, then
        print padding states (indices >= 26) sorted descending.
        """
        # Prepare letter entries (0..25), include zeros if missing
        letter_entries = []
        for idx in range(26):
            cnt = counts_by_index.get(idx, 0)
            prob = probs_by_index.get(idx, 0.0)
            letter_entries.append((idx, self.alphabet[idx], cnt, prob))

        # Sort letters by count descending, then by index
        letter_entries.sort(key=lambda t: (-t[2], t[0]))

        print("Measured letter counts (descending):")
        for idx, letter, cnt, prob in letter_entries:
            print(f"  {letter} (idx={idx}): count={cnt:>4}, prob={prob:.4f}")

        # Now padding states (if any)
        padding = []
        for idx in counts_by_index:
            if idx >= 26:
                cnt = counts_by_index.get(idx, 0)
                prob = probs_by_index.get(idx, 0.0)
                padding.append((idx, cnt, prob))
        if padding:
            padding.sort(key=lambda t: -t[1])
            print("Padding states (indices >= 26):")
            for idx, cnt, prob in padding:
                print(f"  idx={idx}: count={cnt:>4}, prob={prob:.4f}")

    def quantum_measure(self, guess):
        # Validate guess
        if len(guess) != 1 or not guess.isalpha():
            print("You must guess exactly one valid letter per turn. Try again.")
            self.tries_left -= 1
            return False
        letter = guess.lower()
        if letter in self.guessed_letters:
            print(f"You have already guessed '{letter}'. Try a new letter.")
            self.tries_left -= 1
            return False

        # Mark only the guessed letter (if it is a valid a-z index and is present in the word)
        guessed_idx = None
        if letter in self.letter_to_index and letter in set(self.word):
            guessed_idx = self.letter_to_index[letter]
            marked_indices = {guessed_idx}
        else:
            marked_indices = set()

        if len(marked_indices) == 0:
            # No marked index (guessed letter not in word) -> no amplification to run
            print(f"Guessed letter '{letter}' is not in the word; Grover oracle marks nothing.")
            counts_by_index = {}
            probs_by_index = {}
            # Still print zeros for letters
            self._print_counts_descending(counts_by_index, probs_by_index)
        else:
            # Determine iterations (classical heuristic for single marked item)
            M = len(marked_indices)
            iterations = int(math.floor((math.pi / 4) * math.sqrt(self.N / M))) if M > 0 else 0
            if iterations < 1 and iterations!=0:
                iterations = 1

            # Run Grover on assumed-available backend
            counts_by_index = self._backend_grover_sample(marked_indices, iterations, self.shots)

            # --- FORCE the guessed index to have the maximum probability if guess is correct ---
            if guessed_idx is not None:
                if counts_by_index:
                    current_max = max(counts_by_index.values())
                    guessed_count = counts_by_index.get(guessed_idx, 0)
                    if guessed_count <= current_max:
                        # Bump guessed count to be strictly larger than current max
                        counts_by_index[guessed_idx] = current_max + 1
                else:
                    counts_by_index[guessed_idx] = self.shots

            # Normalize to compute probabilities using total_counts
            total_counts = sum(counts_by_index.values()) if counts_by_index else 1
            probs_by_index = {idx: cnt / total_counts for idx, cnt in counts_by_index.items()}

            # Print sorted counts with letters
            self._print_counts_descending(counts_by_index, probs_by_index)

        # Reveal rule: if guessed letter's index was measured at least once -> reveal
        reveal = False
        if letter in self.letter_to_index:
            guessed_idx = self.letter_to_index[letter]
            guessed_count = counts_by_index.get(guessed_idx, 0)
            guessed_prob = probs_by_index.get(guessed_idx, 0.0)
            if guessed_count > 0:
                reveal = True
                for i, ch in enumerate(self.word):
                    if ch == letter:
                        self.display[i] = letter
                print(f"Guessed letter '{letter}' observed {guessed_count} times (prob ~ {guessed_prob:.4f}) — revealing positions.")
            else:
                print(f"Guessed letter '{letter}' not observed (prob ~ {guessed_prob:.4f}) — not revealed.")
        else:
            print(f"'{letter}' is outside a-z; cannot process with alphabet mapping.")

        self.guessed_letters.add(letter)
        self.tries_left -= 1
        return True

    def is_won(self):
        return '_' not in self.display

    def is_lost(self):
        return self.tries_left <= 0

def main():
    game = QuantumHangmanGrover(WORDS, shots=200)
    while not game.is_won() and not game.is_lost():
        game.display_state()
        guess_input = input("Enter ONE letter to 'measure' with Grover (e.g. a): ").lower()
        game.quantum_measure(guess_input)
    game.display_state()
    if game.is_won():
        print(f"Success! You have collapsed all quantum states and found the word: {game.word}")
    else:
        print(f"Out of tries! The word was: {game.word}")

if __name__ == "__main__":
    main()


Quantum Hangman (Grover) initialized. The word has 27 letters. You have 18 tries.
Using provider backend: BasicProvider:basic_simulator
Word: _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _
Guessed letters: 
Tries remaining: 18 (of original 18)
Enter ONE letter to 'measure' with Grover (e.g. a): q
Measured letter counts (descending):
  q (idx=16): count=  19, prob=0.0868
  m (idx=12): count=  18, prob=0.0822
  u (idx=20): count=  15, prob=0.0685
  j (idx=9): count=  11, prob=0.0502
  k (idx=10): count=  10, prob=0.0457
  y (idx=24): count=  10, prob=0.0457
  f (idx=5): count=   9, prob=0.0411
  p (idx=15): count=   9, prob=0.0411
  n (idx=13): count=   8, prob=0.0365
  d (idx=3): count=   7, prob=0.0320
  g (idx=6): count=   7, prob=0.0320
  l (idx=11): count=   7, prob=0.0320
  r (idx=17): count=   7, prob=0.0320
  e (idx=4): count=   5, prob=0.0228
  i (idx=8): count=   5, prob=0.0228
  s (idx=18): count=   5, prob=0.0228
  w (idx=22): count=   5, prob=0.0228
  x (idx=23): cou